# EOPF Zarr GDAL Plugin Testing

This notebook tests the EOPF Zarr GDAL plugin functionality and documents known issues with GitHub Actions artifacts.

## Known Issues with GitHub Actions Artifacts

### Windows
- **Error 126**: "Can't load requested DLL: gdal_EOPFZarr.dll - Module could not be found"
- **Status**: Plugin loads with error but still reads Zarr files correctly
- **Root Cause**: Dependency issues with GitHub Actions build environment

### macOS (M-series processors)
- **Error**: Architecture mismatch - GitHub Actions builds x86_64, but M-series Macs need arm64
- **Example Error**: 
  ```
  mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e' or 'arm64')
  ```
- **Solution**: Need to build specifically for arm64 architecture or provide universal binaries

### Linux
- Status to be tested

## Test Strategy
1. Test plugin registration
2. Test with sample Zarr data
3. Document errors but verify if functionality still works
4. Compare local build vs GitHub Actions artifacts

In [ ]:
import os
import sys
import platform
from osgeo import gdal

def check_plugin_status():
    """Comprehensive check of EOPF Zarr plugin status"""
    print("=== EOPF Zarr Plugin Diagnostic ===")
    print(f"Platform: {platform.system()} {platform.machine()}")
    print(f"GDAL Version: {gdal.VersionInfo()}")
    print(f"Python Version: {sys.version}")
    print()
    
    # Check if plugin is registered
    print("1. Checking plugin registration...")
    driver = gdal.GetDriverByName('EOPFZARR')
    if driver is not None:
        print("   ✓ EOPFZARR driver is registered")
        print(f"   Driver description: {driver.GetDescription()}")
        print(f"   Driver metadata: {driver.GetMetadata()}")
    else:
        print("   ✗ EOPFZARR driver not found!")
        print("   Available drivers containing 'zarr':")
        for i in range(gdal.GetDriverCount()):
            drv = gdal.GetDriver(i)
            if 'zarr' in drv.GetDescription().lower():
                print(f"     - {drv.GetDescription()}")
    
    print()
    
    # Check GDAL driver path
    print("2. Checking GDAL driver path...")
    driver_path = os.environ.get('GDAL_DRIVER_PATH', 'Not set')
    print(f"   GDAL_DRIVER_PATH: {driver_path}")
    
    if driver_path != 'Not set' and os.path.exists(driver_path):
        print(f"   Plugin files in {driver_path}:")
        for file in os.listdir(driver_path):
            if 'eopf' in file.lower() or 'zarr' in file.lower():
                print(f"     - {file}")
    
    print()
    return driver is not None

# Run the diagnostic
plugin_available = check_plugin_status()

## Test Plugin with Sample Data

Now let's test if the plugin can actually read Zarr data, even if there are loading errors.

In [ ]:
from osgeo import gdal
import os

# Test with sample Zarr dataset
zarr_file = "S01SIWGRH_20240201T164915_0025_A146_S750E_VH.zarr"

print(f"Attempting to open: {zarr_file}")

# Check if file exists
if os.path.exists(zarr_file):
    print(f"✓ File exists: {zarr_file}")
    dataset = gdal.OpenEx(zarr_file, gdal.OF_MULTIDIM_RASTER)
    if dataset:
        print("✓ Dataset opened successfully")
        print(f"  Driver: {dataset.GetDriver().GetDescription()}")
    else:
        print("✗ Failed to open dataset")
        dataset = None
else:
    print(f"✗ File not found: {zarr_file}")
    print("   Let's try the sample data in the tests directory...")
    
    # Try sample data from tests
    sample_data_path = "../tests/sample_data"
    if os.path.exists(sample_data_path):
        print(f"✓ Found sample data at: {sample_data_path}")
        dataset = gdal.OpenEx(sample_data_path, gdal.OF_MULTIDIM_RASTER)
        if dataset:
            print("✓ Sample dataset opened successfully")
            print(f"  Driver: {dataset.GetDriver().GetDescription()}")
        else:
            print("✗ Failed to open sample dataset")
            dataset = None
    else:
        print(f"✗ Sample data not found at: {sample_data_path}")
        dataset = None

print(f"\nFinal dataset status: {'Available' if dataset else 'Not available'}")

## Get the root group of the dataset

In [ ]:
if dataset is not None:
    print("Getting root group...")
    root_group = dataset.GetRootGroup()
    if root_group:
        print("✓ Root group obtained successfully")
        print(f"  Group name: {root_group.GetName()}")
    else:
        print("✗ Failed to get root group")
        root_group = None
else:
    print("⚠ No dataset available - skipping root group test")
    root_group = None

## To access attributes, loop through the attributes

In [ ]:
if root_group is not None:
    print("Reading attributes...")
    attributes = root_group.GetAttributes()
    if attributes:
        print(f"Found {len(attributes)} attributes:")
        for attribute in attributes:
            print(f"  {attribute.GetName()}: {attribute.ReadAsString()}")
    else:
        print("No attributes found")
else:
    print("⚠ No root group available - skipping attributes test")

other_metadata: { "azimuth_steering_rate": 1.590368784, "downlink_information": { "iw1": { "azimuth_time": "2024-02-01_t16:49:13.204896", "bit_error_count": { "num_err_azimuth_beam_address": 0, "num_err_baq_block_length": 0, "num_err_baq_mode": 0, "num_err_cal_mode": 0, "num_err_cal_type": 0, "num_err_calibration_beam_address": 0, "num_err_data_take_id": 0, "num_err_ecc_number": 0, "num_err_elevation_beam_address": 0, "num_err_instrument_config_id": 0, "num_err_number_of_quads": 0, "num_err_packet_count": 0, "num_err_polarisation": 0, "num_err_pri": 0, "num_err_pri_count": 0, "num_err_range_decimation": 0, "num_err_rank": 0, "num_err_rx_channel_id": 0, "num_err_rx_gain": 0, "num_err_sas_test_mode": 0, "num_err_signal_type": 0, "num_err_ssb_error_flag": 0, "num_err_swap_flag": 0, "num_err_swath_number": 0, "num_err_swl": 0, "num_err_swst": 0, "num_err_sync_marker": 0, "num_err_temp_comp": 7045, "num_err_test_mode": 0, "num_err_tx_pulse_number": 0, "num_err_tx_pulse_start_frequency": 0, 

In [ ]:
# Summary: Is the plugin "working"?
print("=== PLUGIN STATUS SUMMARY ===")
print()

if plugin_available:
    print("✓ Plugin is registered with GDAL")
    
    if dataset is not None:
        print("✓ Plugin can read Zarr datasets")
        print("✓ Plugin appears to be functionally working")
        print()
        print("CONCLUSION: Even if there are loading errors (like Error 126 on Windows")
        print("or architecture mismatches on macOS), the plugin IS WORKING if it can:")
        print("  1. Register with GDAL")
        print("  2. Successfully read Zarr files")
        print("  3. Extract metadata and attributes")
        print()
        print("💡 RECOMMENDATION:")
        print("   Consider this plugin 'working' for users despite loading errors.")
        print("   The errors are cosmetic and don't prevent functionality.")
    else:
        print("⚠ Plugin registered but cannot read test data")
        print("   This could be due to missing test files rather than plugin issues")
else:
    print("✗ Plugin is not registered")
    print("   This indicates a real problem that needs to be fixed")

print()
print("For the GitHub Actions artifacts:")
print("- Windows: Works with Error 126 (acceptable)")
print("- macOS M-series: Architecture mismatch (needs arm64 build)")
print("- Linux: Status unknown (needs testing)")